In [1]:
import numpy as np
import pandas as pd

In [60]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv",usecols=['user_id','business_id','stars_x','text','city','latitude','longitude','review_count','categories','name'],nrows=100000,keep_default_na=False)

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [33]:
df2=df1.iloc[:20000,:]

In [143]:
df1.head(3)

,user_id,business_id,stars_x,text,name,city,latitude,longitude,review_count,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,North Wales,40.210196,-75.223639,169,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,Philadelphia,39.952103,-75.172753,144,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,Tucson,32.207233,-110.980864,47,"Restaurants, Breakfast & Brunch"


In [62]:
pivot_table_df=pd.pivot_table(df1,index=['name'],columns=['user_id'],values='stars_x')

In [63]:
pivot_table_df.fillna(value=0,inplace=True)

In [64]:
pivot_table_df

user_id,---2PmXbF47D870stH1jqA,--4AjktZiHowEIBCMd4CZA,--E0uVPphTORm_OiZ5KCvA,--KMTwCrhKKUmr7riuS4WQ,--S8M395r8NtOCvS2LRfDw,--UizzbnQlZg7bEv2oXEyg,--WhhlIim6ISY5yhFYZreg,--_H9j6ggxvqhh9nPofZwg,--_r6E98SNIrGU7weyNxbw,--ccVMj2PN6Z9qtdOdlung,...,zzeRzizkihWHz9bVAvbcVw,zzgLgszu48af4NJULymzAA,zzgMuJgxmToqcJ5iu1TngQ,zziJLt25YU6dp01sewR-IQ,zzjThJ4A1m1N78gquSR_QA,zzljYL9c22HWRO3VvJ1_Pg,zzsPf8xNW11nd0B6MZqfRw,zzsqjDvanJhH9tn8NautOQ,zztkCqqgR6VntYbqio4UTQ,zzxUT-fFNXpp5i5xJHmvpA
name,,,,,,,,,,,,,,,,,,,,,
"""No Macarena"" DJ & Custom Music Service",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'feine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+1 Gaming,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 Tippling Place,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-275 Rest Area Manatee County Mile 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uBreakiFix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
velofix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vybe urgent care,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from scipy.sparse import csr_matrix

In [66]:
df_sparse=csr_matrix(pivot_table_df)

In [67]:
from sklearn.neighbors import NearestNeighbors
model1=NearestNeighbors(algorithm='brute')

In [68]:
model1.fit(df_sparse)

NearestNeighbors(algorithm='brute')

In [135]:
def recommend(ind):
    distanceS,recS=model1.kneighbors(pivot_table_df.iloc[ind,:].values.reshape(1,-1),n_neighbors=20)
    for i in range(len(recS[0])):
        print(pivot_table_df.index[recS[0][i]])

In [136]:
recommend(1000)

Boise International Market
All 4 Dance
ADT Security Services
3am Liquor
Associated Dental Care - Tucson N Campbell
A.B. Collier Wrecker Service
Arch Engraving
A1 RV Sales
Atlantic Vapor
Aqua-Tots Swim Schools - Eagle
American Family Care
Action Towing
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
Accurate Window & Screen
A Psychic and Tarot Card Readings
Amy & Barber Beauty Salon
Aloha Japanese Steakhouse
Affordable Lock & Security Solutions - Tampa
Amor Nail and Spa


In [138]:
recommend(0)

"No Macarena" DJ & Custom Music Service
All 4 Dance
ADT Security Services
A1 RV Sales
Arch Engraving
American Family Care
Affordable Lock & Security Solutions - Tampa
A Psychic and Tarot Card Readings
Associated Dental Care - Tucson N Campbell
Aqua-Tots Swim Schools - Eagle
Action Towing
Accurate Window & Screen
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
A.B. Collier Wrecker Service
3am Liquor
Atlantic Vapor
Aloha Japanese Steakhouse
Amor Nail and Spa
Amy & Barber Beauty Salon


In [142]:
df1['text']=df1['text']+df1['categories']

In [149]:
df1['categories'][886]

'Bars, Nightlife, Lounges, Cocktail Bars'

## Content based recommendation

In [5]:
vdf=pd.read_json("../r2Data/yelp/yelp_academic_dataset_review.json",nrows=200000,lines=True)

In [22]:
udf=pd.read_json('../r2Data/yelp/yelp_academic_dataset_business.json',lines=True,nrows=10000)

In [23]:
udf.drop(columns=['business_id','address',"state",'postal_code','stars','review_count','is_open','hours'],inplace=True)

In [24]:
udf

,name,city,latitude,longitude,attributes,categories
0,"Abby Rappoport, LAC, CMQ",Santa Barbara,34.426679,-119.711197,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop..."
1,The UPS Store,Affton,38.551126,-90.335695,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."
2,Target,Tucson,32.223236,-110.880452,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G..."
3,St Honore Pastries,Philadelphia,39.955505,-75.155564,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,Perkiomen Valley Brewery,Green Lane,40.338183,-75.471659,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
...,...,...,...,...,...,...
9995,Knuckle Dusters Vapor,Reno,39.505271,-119.785091,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Tobacco Shops, Shopping, Vape Shops"
9996,Ricky's All Day Grill,St Albert,53.620257,-113.611680,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Canadian (New), American (New), Restaurants"
9997,Admirals Club PHL Terminal F,Philadelphia,39.880694,-75.237370,"{'Alcohol': ''full_bar'', 'HappyHour': 'False'...","Bars, Nightlife, Airport Lounges"
9998,Las Palmas Mexican Restaurant,Nashville,36.152919,-86.798455,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, Mexican"


## Content based model 1

using only udf

In [25]:
from nltk.stem.porter import PorterStemmer

In [26]:
ps=PorterStemmer()
def stemData(text):
    texts=[]
    for word in text.split():
        texts.append(ps.stem(word))
    return " ".join(texts)

In [27]:
udf.drop(columns=['attributes'],inplace=True)

In [28]:
udf['categories'][0]

'Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists'

In [29]:
udf.columns

Index(['name', 'city', 'latitude', 'longitude', 'categories'], dtype='object')

In [30]:
udf.dropna(inplace=True)

In [31]:
udf['categories']=udf['categories'].apply(lambda x : x.lower().replace("& ","").replace("&","").replace("  "," ").replace(",",""))

In [32]:
udf['categories']=udf['categories'].apply(lambda x : stemData(x))

In [45]:
udf.head(10)

,name,city,latitude,longitude,categories
0,"Abby Rappoport, LAC, CMQ",Santa Barbara,34.426679,-119.711197,doctor tradit chines medicin naturopathic/holi...
1,The UPS Store,Affton,38.551126,-90.335695,ship center local servic notari mailbox center...
2,Target,Tucson,32.223236,-110.880452,depart store shop fashion home garden electron...
3,St Honore Pastries,Philadelphia,39.955505,-75.155564,restaur food bubbl tea coffe tea bakeri
4,Perkiomen Valley Brewery,Green Lane,40.338183,-75.471659,brewpub breweri food
5,Sonic Drive-In,Ashland City,36.269593,-87.058943,burger fast food sandwich food ice cream froze...
6,Famous Footwear,Brentwood,38.627695,-90.340465,sport good fashion shoe store shop sport wear ...
7,Temple Beth-El,St. Petersburg,27.766590,-82.732983,synagogu religi organ
8,Tsevi's Pub And Grill,Affton,38.565165,-90.321087,pub restaur italian bar american (traditional)...
9,Sonic Drive-In,Nashville,36.208102,-86.768170,ice cream frozen yogurt fast food burger resta...


# using sklearn

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
cv=CountVectorizer(stop_words='english',max_features=1000)

In [35]:
vectors=cv.fit_transform(udf['categories']).toarray()

In [36]:
len(cv.get_feature_names_out())

1000

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
cos_vecotrs=cosine_similarity(vectors)

In [41]:
cos_vecotrs[1000]

array([0.        , 0.        , 0.        , ..., 0.        , 0.40824829,
       0.        ])

In [42]:
def getBestHotelMatch(index):
    matchIndex=cos_vecotrs[index]
    lst=sorted(list(enumerate(cos_vecotrs[0])),key=lambda x : x[1],reverse=True)
    lst=lst[1:20]
    return lst

In [43]:
getBestHotelMatch(0)

[(6649, np.float64(0.7745966692414835)),
 (7060, np.float64(0.7745966692414835)),
 (7463, np.float64(0.7745966692414835)),
 (9773, np.float64(0.7745966692414835)),
 (5857, np.float64(0.7115124735378854)),
 (6511, np.float64(0.7)),
 (6060, np.float64(0.6708203932499369)),
 (8088, np.float64(0.6708203932499369)),
 (3341, np.float64(0.6454972243679029)),
 (2877, np.float64(0.6324555320336759)),
 (6471, np.float64(0.6324555320336759)),
 (2033, np.float64(0.6139406135149206)),
 (5596, np.float64(0.6139406135149206)),
 (547, np.float64(0.5720775535473555)),
 (9581, np.float64(0.5715476066494082)),
 (146, np.float64(0.565685424949238)),
 (5296, np.float64(0.565685424949238)),
 (7694, np.float64(0.565685424949238)),
 (458, np.float64(0.5477225575051663))]

In [44]:
udf.iloc[6649]

name                                   Vitality Acupuncture
city                                               Meridian
latitude                                          43.603609
longitude                                       -116.412797
categories    health medic acupunctur tradit chines medicin
Name: 6655, dtype: object

## Content based model 2

In [67]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=50,random_state=42)
km.fit(vectors)

KMeans(n_clusters=50, random_state=42)

In [68]:
km.labels_

array([24, 49, 17, ..., 10,  1,  3], dtype=int32)

In [70]:
from sklearn.neighbors import NearestNeighbors
NN=NearestNeighbors(n_neighbors=50,metric='cosine')

In [71]:
NN.fit(vectors)

NearestNeighbors(metric='cosine', n_neighbors=50)

In [72]:
cluster=pd.DataFrame({'Cluster': km.labels_})

In [75]:
udf['Cluster']=cluster

In [76]:
udf.sample(10)

,name,city,latitude,longitude,categories,Cluster
4223,Siam Thai Restaurant,Boise,43.579153,-116.175362,thai restaur,45.0
7116,Clock Restaurant,Zephyrhills,28.234519,-82.182502,restaur breakfast brunch diner,30.0
7023,Winn Dixie,Chalmette,29.948702,-89.960722,groceri food,35.0
3060,Bistro66,Pinellas Park,27.847022,-82.728979,restaur italian german american (new),33.0
4838,La Maison Santa Barbara,Santa Barbara,34.417109,-119.699329,hotel travel vacat rental event plan servic hotel,47.0
3523,Willis Honda,Burlington,40.085072,-74.835083,car dealer auto repair automot,28.0
7031,Mandala Massage Supply & Apothecary,Reno,39.512646,-119.802510,massag health medic cosmet beauti suppli shop ...,19.0
1942,Mainland Grille,Harleysville,40.248261,-75.360939,burger venu event space diner restaur event pl...,48.0
7119,Domino's Pizza,Spring Hill,28.467735,-82.510825,restaur pizza sandwich chicken wing,40.0
5451,Banner Urgent Care,Tucson,32.237169,-110.858924,doctor urgent care health medic emerg room,3.0


In [81]:
def recommendData(text):
    text_vector=cv.transform([text])
    clusterIndex=km.predict(text_vector)[0]
    cluster_indices = udf.index[udf['Cluster'] == clusterIndex].tolist()
    cluster_vectors = vectors[cluster_indices]
    NN.fit(cluster_vectors)
    
    # Find nearest neighbors
    distances, indices = NN.kneighbors(text_vector)
    recommendations = udf.iloc[[cluster_indices[i] for i in indices[0]]]['name']
    return recommendations

### If the queried word is found it will give mp accurate resuts otherwise some unepected results

In [90]:
recommendData('food')

5142                               MexiPhilly Food Cart
4764      Audubon Aquarium of the Americas - Food Court
70                                        Mr. Margarita
1737                                     Feed the Camel
2410                               Sunflower Truck Stop
2699                           B and D's Curbside Grill
1989                            Big Bad Wolf Food Truck
1863                                      JJ's Barbeque
3032                             Battle Born Food Truck
673                                Mount Mogrit Gourmet
3733                       Driving Winds Blueberry Farm
7471                                      P & S Ravioli
1922                                        Cup of Corn
5427                             Little Road Farm Stand
93                                              Impasto
7035                            Mis Antojitos Mexicanos
2162                                     Chef Johnson's
6694                               Brawley's Res